# Goal: a scene classification model that classifies an image to one of 6 labels:
- Buildings
- Forests
- Mountains
- Glacier
- Street
- Sea

Dataset: https://www.kaggle.com/datasets/nitishabharathi/scene-classification?resource=download&select=train-scene+classification

In [ ]:
# Import the libraries we'll use below.
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

In [ ]:
np.random.seed(seed=123)

## Load data

In [ ]:
from PIL import Image

def generate_xy(df, resize=False):
    X = []
    Y = []
    for i in df.iterrows():
        img_name = i[1]['image_name']
        img = Image.open(f'train-scene classification/train/{img_name}')
        img_arr = np.array(img)
        if img_arr.shape[0] == 150:
            X.append(np.array(img))
            Y.append(i[1]['label'])
        elif resize:
            X.append(np.array(img.resize((150,150))))
            Y.append(i[1]['label'])
        else:
            continue

    X = np.array(X)
    Y = np.array(Y)
    
    return X, Y

In [ ]:
labels = ['building','forest','glacier','mountain',
            'sea','street']

In [ ]:
samples_df = pd.read_csv('train-scene classification/train.csv')

In [ ]:
classes = samples_df['label'].unique()
classes

In [ ]:
dfs = [] 
for i in classes:
    dfs.append(samples_df[samples_df.label==i].sample(n=2500))

sub_samples_df = pd.concat(dfs)
sub_samples_df.shape


In [ ]:
val_df = sub_samples_df.sample(frac=.2)
test_df = sub_samples_df[~sub_samples_df.index.isin(val_df.index)].sample(frac=.2)
train_df = sub_samples_df[~sub_samples_df.index.isin(val_df.index) & ~sub_samples_df.index.isin(test_df.index)]

##### Generate training data

In [ ]:
train_X, train_Y = generate_xy(train_df)
val_X, val_Y = generate_xy(val_df)
test_X, test_Y = generate_xy(test_df)

In [ ]:
shuffle = tf.random.shuffle(tf.range(tf.shape(train_X)[0], dtype=tf.int32))
train_X = tf.gather(train_X, shuffle).numpy()
train_Y = tf.gather(train_Y, shuffle).numpy()

shuffle = tf.random.shuffle(tf.range(tf.shape(val_X)[0], dtype=tf.int32))
val_X = tf.gather(val_X, shuffle).numpy()
val_Y = tf.gather(val_Y, shuffle).numpy()

## Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Explore image specifics

In [ ]:
print(f'total training samples X: {train_X.shape[0]}')
print(f'total training samples Y: {train_Y.shape[0]}')
print(f'total val samples X: {val_X.shape[0]}')
print(f'total val samples Y: {val_Y.shape[0]}')
print(f'total test samples X: {test_X.shape[0]}')
print(f'total test samples Y: {test_Y.shape[0]}')
print(f'image size: {train_X.shape[1:3]}')

### Check class balance across train/val dataset

#### Whole dataset

In [ ]:
class_counts = samples_df.groupby(by='label').count().reset_index()
class_counts.rename(columns={'image_name':'counts'}, inplace=True)
sns.barplot(x='label', y='counts', data=class_counts) 

#### Train dataset

In [ ]:
train_class_counts = train_df.groupby(by='label').count().reset_index()
train_class_counts.rename(columns={'image_name':'counts'}, inplace=True)
sns.barplot(x='label', y='counts', data=train_class_counts) 

#### Val dataset

In [ ]:
val_class_counts = val_df.groupby(by='label').count().reset_index()
val_class_counts.rename(columns={'image_name':'counts'}, inplace=True)
sns.barplot(x='label', y='counts', data=val_class_counts) 

#### Test dataset

In [ ]:
test_class_counts = test_df.groupby(by='label').count().reset_index()
test_class_counts.rename(columns={'image_name':'counts'}, inplace=True)
sns.barplot(x='label', y='counts', data=test_class_counts) 

## Image preprocessing

In [ ]:
train_X_norm = []
for image in train_X:
    image = image.astype('float64')
    image *= 255.0/image.max()
    image = image.astype('uint8')
    train_X_norm.append(image)

train_X_norm = np.array(train_X_norm)

val_X_norm = []
for image in val_X:
    image = image.astype('float64')
    image *= 255.0/image.max()
    image = image.astype('uint8')
    val_X_norm.append(image)

val_X_norm = np.array(val_X_norm)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(ncols=3,nrows=1,figsize=(16,6))

bands = train_X.transpose(3, 0, 1, 2)
axes[0].set_title('band 0')
sns.histplot(np.random.choice(bands[0].ravel(), size=1000), ax=axes[0])
axes[1].set_title('band 1')
sns.histplot(np.random.choice(bands[1].ravel(), size=1000), ax=axes[1])
axes[2].set_title('band 2')
sns.histplot(np.random.choice(bands[2].ravel(), size=1000), ax=axes[2])

plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=3,nrows=1,figsize=(16,6))

bands = train_X_norm.transpose(3, 0, 1, 2)
axes[0].set_title('band 0')
sns.histplot(np.random.choice(bands[0].ravel(), size=1000), ax=axes[0])
axes[1].set_title('band 1')
sns.histplot(np.random.choice(bands[1].ravel(), size=1000), ax=axes[1])
axes[2].set_title('band 2')
sns.histplot(np.random.choice(bands[2].ravel(), size=1000), ax=axes[2])

plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=2,nrows=1,figsize=(16,6))

axes[0].imshow(train_X[0])
axes[1].imshow(train_X_norm[0])

## Train Model

### Train Feed Forward Network

In [ ]:
def build_ffn_model(n_classes,
                input_shape,
                hidden_layer_sizes=[],
                activation='relu',
                optimizer='SGD',
                learning_rate=0.001):
    """Build a multi-class logistic regression model using Keras.

    Args:
    n_classes: Number of output classes in the dataset.
    hidden_layer_sizes: A list with the number of units in each hidden layer.
    activation: The activation function to use for the hidden layers.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The desired learning rate for the optimizer.

    Returns:
    model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)

    model = keras.Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=input_shape))
    for layer_size in hidden_layer_sizes:
        
        model.add(keras.layers.Dense(
          units=layer_size,
          activation=activation,
        ))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'
    ))
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate) if optimizer == 'SGD' \
             else tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    return model

In [ ]:
model_ffn = build_ffn_model(n_classes=len(classes),
                    input_shape=train_X.shape,
                    hidden_layer_sizes=[150],
                    activation='relu',
                    optimizer='Adam')

In [ ]:
print('Training...')
n_epochs = 3
history_ffn = model_ffn.fit(
    x=train_X,
    y=train_Y,
    epochs=n_epochs,
    batch_size=16,
    validation_data=(val_X, val_Y),
    verbose=True)

In [ ]:
train_accuracy = history_ffn.history['accuracy']
val_accuracy = history_ffn.history['val_accuracy']
plt.plot(train_accuracy, label='train_accuracy')
plt.plot(val_accuracy, label='validation accuracy')
plt.xticks(range(n_epochs))
plt.xlabel('Train epochs')
plt.legend()
plt.show()

model_eval = model_ffn.evaluate(x=test_X, y=test_Y, verbose=0,
                             return_dict=True)

test_accuracy = model_eval['accuracy']
print(test_accuracy)

### Predictions

In [ ]:
predictions = model_ffn.predict(test_X)

In [ ]:
def view_predictions(samples, actual, predictions):
    fig, axes = plt.subplots(nrows=3, ncols=3)
    fig.set_size_inches(10, 12)

    for i in range(3):
        for j in range(3):
            s = i+j
            scores = predictions[s]
            guess = np.where(scores==scores.max())
            axes[i][j].imshow(samples[s])
            axes[i][j].set_title('ACTUAL: '+str(labels[actual[s]])+ 
                        ' PRED: '+str(labels[guess[0][0]]))

    plt.show()

In [ ]:
view_predictions(test_X, test_Y, predictions)

### Train 2d CNN

In [ ]:
def build_2d_cnn_model(n_classes, input_shape,
                optimizer='SGD',
                learning_rate=0.001,
                padding='same'):
    """Build a multi-class 2D CNN using Keras.

    Args:
    n_classes: Number of output classes in the dataset.
    input_shape: The shape of the input dataset.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The desired learning rate for the optimizer.

    Returns:
    model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)

    model = keras.Sequential([
      keras.layers.Rescaling(1./255, input_shape=input_shape),
      keras.layers.Conv2D(16, 3, padding=padding, activation='relu'),
      keras.layers.MaxPooling2D(),
      keras.layers.Conv2D(32, 3, padding=padding, activation='relu'),
      keras.layers.MaxPooling2D(),
      keras.layers.Conv2D(64, 3, padding=padding, activation='relu'),
      keras.layers.MaxPooling2D(),
      keras.layers.Flatten(),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dropout(rate=0.5),
      # keras.layers.Dense(n_classes),
    ])
    
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'
    ))
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate) if optimizer == 'SGD' \
             else tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    return model


In [ ]:

input_shape = train_X.shape[1:]
model_2d = build_2d_cnn_model(n_classes=len(classes),
                      input_shape=input_shape,
                      optimizer='Adam')

In [ ]:
model_2d.build()

print(model_2d.summary())

In [ ]:
print('Training...')
n_epochs = 5
cnn2d_history = model_2d.fit(
    x=train_X,
    y=train_Y,
    epochs=n_epochs,
    validation_data=(val_X, val_Y),
    verbose=True)

In [ ]:
def print_curves(history, n_epochs):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(n_epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

print_curves(cnn2d_history, n_epochs)

### Predictions

In [ ]:
model_eval = model_2d.evaluate(x=test_X, y=test_Y, verbose=0,
                             return_dict=True)

test_accuracy = model_eval['accuracy']
print(test_accuracy)

In [ ]:
predictions = model_2d.predict(test_X)

In [ ]:
view_predictions(test_X, test_Y, predictions)

# Cross Validation on CNN 2D

In [ ]:
images, labels = generate_xy(sub_samples_df)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

for kfold, (train, test) in enumerate(KFold(n_splits=3, shuffle=True).split(images, labels)):
    # clear the session 
    tf.keras.backend.clear_session()

    # calling the model and compile it 
    model = build_2d_cnn_model(len(np.unique(labels[train])),images[train].shape[1:])

    print(f'Iteration {kfold}')

    # run the model 
    epochs = 10
    hist = model.fit(images[train], 
              labels[train],
              batch_size=128, 
              epochs=10, 
              validation_data=(images[test], 
              labels[test]),
              verbose=False
            )
    print_curves(hist, epochs)